In [1]:

import json
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
from IPython.display import display, HTML
#import os
#from numpy.random import randn
#from copy import copy
#from update import update

In [2]:
dataset_filename ='/home/chandra/WORK/Prescription_Based_Prediction/predBasedPres_corrected_50.jsonl'
df = pd.DataFrame()
data = ''
with open(dataset_filename) as json_data:
        data = json.load(json_data)
        df= json_normalize(data)
json_data.close()


In [3]:
df.shape

(50, 403)

In [4]:
df.describe()

,cms_prescription_counts.ABACAVIR,cms_prescription_counts.ABILIFY,cms_prescription_counts.ACTONEL,cms_prescription_counts.ACYCLOVIR,cms_prescription_counts.ADVAIR DISKUS,cms_prescription_counts.ALBUTEROL SULFATE,cms_prescription_counts.ALENDRONATE SODIUM,cms_prescription_counts.ALLOPURINOL,cms_prescription_counts.ALPRAZOLAM,cms_prescription_counts.AMANTADINE,...,cms_prescription_counts.WARFARIN SODIUM,cms_prescription_counts.WELCHOL,cms_prescription_counts.XARELTO,cms_prescription_counts.ZEMPLAR,cms_prescription_counts.ZETIA,cms_prescription_counts.ZIPRASIDONE HCL,cms_prescription_counts.ZOLPIDEM TARTRATE,provider_variables.brand_name_rx_count,provider_variables.generic_rx_count,provider_variables.years_practicing
count,2.0,11.000000,1.0,2.000000,4.000000,1.0,3.0,7.000000,15.000000,1.0,...,7.000000,1.0,3.000000,1.0,4.000000,9.000000,17.000000,50.000000,50.000000,50.000000
mean,12.0,128.090909,16.0,192.000000,105.750000,16.0,14.0,33.285714,130.933333,12.0,...,65.857143,13.0,18.666667,12.0,19.000000,49.666667,98.764706,322.760000,1299.640000,5.900000
std,0.0,142.124913,NaN,87.681241,101.700131,NaN,3.0,26.768852,298.570515,NaN,...,54.694389,NaN,6.658328,NaN,7.164728,47.762433,181.858987,796.873241,2234.782076,1.897904
min,12.0,11.000000,16.0,130.000000,23.000000,16.0,11.0,14.000000,14.000000,12.0,...,12.000000,13.0,11.000000,12.0,12.000000,11.000000,11.000000,0.000000,0.000000,1.000000
25%,12.0,29.500000,16.0,161.000000,32.750000,16.0,12.5,16.500000,25.500000,12.0,...,18.000000,13.0,16.500000,12.0,13.500000,18.000000,16.000000,0.000000,84.000000,6.000000
50%,12.0,107.000000,16.0,192.000000,78.000000,16.0,14.0,19.000000,45.000000,12.0,...,76.000000,13.0,22.000000,12.0,18.500000,40.000000,41.000000,64.500000,322.500000,7.000000
75%,12.0,127.000000,16.0,223.000000,151.000000,16.0,15.5,40.500000,77.000000,12.0,...,87.000000,13.0,22.500000,12.0,24.000000,54.000000,92.000000,269.000000,1742.750000,7.000000
max,12.0,427.000000,16.0,254.000000,244.000000,16.0,17.0,86.000000,1200.000000,12.0,...,163.000000,13.0,23.000000,12.0,27.000000,165.000000,783.000000,5175.000000,12701.000000,8.000000


In [5]:
df_sparse =df.fillna(0).to_sparse(fill_value=0)
#list(my_dataframe.columns.values)

In [6]:
print [df_sparse.columns]

[Index([u'cms_prescription_counts.ABACAVIR', u'cms_prescription_counts.ABILIFY',
       u'cms_prescription_counts.ACTONEL',
       u'cms_prescription_counts.ACYCLOVIR',
       u'cms_prescription_counts.ADVAIR DISKUS',
       u'cms_prescription_counts.ALBUTEROL SULFATE',
       u'cms_prescription_counts.ALENDRONATE SODIUM',
       u'cms_prescription_counts.ALLOPURINOL',
       u'cms_prescription_counts.ALPRAZOLAM',
       u'cms_prescription_counts.AMANTADINE',
       ...
       u'cms_prescription_counts.ZIPRASIDONE HCL',
       u'cms_prescription_counts.ZOLPIDEM TARTRATE', u'npi',
       u'provider_variables.brand_name_rx_count', u'provider_variables.gender',
       u'provider_variables.generic_rx_count', u'provider_variables.region',
       u'provider_variables.settlement_type', u'provider_variables.specialty',
       u'provider_variables.years_practicing'],
      dtype='object', length=403)]


**simplifie the column name
<br/>check fro repeating column names
<br/> Check for invalide column name like white spaces etc**

In [7]:
#parts = (label.strip().split('.') for label in df.columns.values)
column_names_list = df.columns.values.tolist()
index_npi = column_names_list.index('npi')
column_names_list.remove('npi')

parts = [lable.strip(' \t\n\r').split('.') for lable in column_names_list] 
parts1 =  [parts_iter[0] for parts_iter in parts]
column_name_new =  [parts_iter[1] for parts_iter in parts]
column_name_new.insert(index_npi,'npi')
#print index_npi
print len(column_name_new)

403


create a dataframe out of parts and use shape to check if there are any rows with more than two values

In [8]:
df_check = pd.DataFrame(parts)
df_check.shape

(402, 2)

There are only 2 columns

create a dataframe out of parts1 and see if all the name are either  'cms_prescription_counts' or 
'provider_variables'. use groupby() to do this


In [9]:
#parts1.append(' ')
df_names_check = pd.DataFrame(parts1, columns=['titles'])
counts = df_names_check.groupby(["titles"]).size()
display (counts)
print "total number of rows is:", len(df_names_check['titles'])
print "cms_prescription_counts + provider_variables", counts['cms_prescription_counts']+counts['provider_variables']

titles
cms_prescription_counts    395
provider_variables           7
dtype: int64

total number of rows is: 402
cms_prescription_counts + provider_variables 402


The sum of number of cms_prescription_counts's and provider_variables's is correct(1 less because we removed 'npi') and we cannot see and invalid entries 
there

In [10]:
print len(column_names_list)

402


now check column_name_new for duplicates and invalid names like ''

In [11]:
#column_name_new.append(' ') #FOR TESTING
#print len(column_name_new)

In [12]:
column_name_new_striped = [x.strip(' \t\n\r') for x in column_name_new]
print len(column_name_new_striped)
#print column_name_new_striped

df_new_names_check = pd.DataFrame(column_name_new_striped, columns=['label']).groupby(['label']).size().reset_index()
df_new_names_check.columns = ['label', 'count']

display (df_new_names_check.query("count != 1" ))
display (df_new_names_check.query(("label == ''")))
display (df_new_names_check.query("label == ' '"))


df_new_names_check.mean()

403


,label,count


,label,count


,label,count


count    1.0
dtype: float64

The mean is 1.0 which mean all the labels appear only once and there are no duplicates also there are no empty string for column names 

**Now remove the cms_prescription_counts and provider_variables from the titles and have a cleaner names

replace the column name inplace uisng the lists column_names_list and column_name_new_striped

In [13]:
column_names_list.insert(index_npi,'npi')


In [14]:
#df.columns = column_name_new_striped
new_names_dict = {}
for i in range(0,403):
    new_names_dict[column_names_list[i]] =  column_name_new_striped[i]
print len(new_names_dict)

403


In [15]:
new_names_dict

{u'cms_prescription_counts.ABACAVIR': u'ABACAVIR',
 u'cms_prescription_counts.ABILIFY': u'ABILIFY',
 u'cms_prescription_counts.ACTONEL': u'ACTONEL',
 u'cms_prescription_counts.ACYCLOVIR': u'ACYCLOVIR',
 u'cms_prescription_counts.ADVAIR DISKUS': u'ADVAIR DISKUS',
 u'cms_prescription_counts.ALBUTEROL SULFATE': u'ALBUTEROL SULFATE',
 u'cms_prescription_counts.ALENDRONATE SODIUM': u'ALENDRONATE SODIUM',
 u'cms_prescription_counts.ALLOPURINOL': u'ALLOPURINOL',
 u'cms_prescription_counts.ALPRAZOLAM': u'ALPRAZOLAM',
 u'cms_prescription_counts.AMANTADINE': u'AMANTADINE',
 u'cms_prescription_counts.AMILORIDE HCL': u'AMILORIDE HCL',
 u'cms_prescription_counts.AMIODARONE HCL': u'AMIODARONE HCL',
 u'cms_prescription_counts.AMITIZA': u'AMITIZA',
 u'cms_prescription_counts.AMITRIPTYLINE HCL': u'AMITRIPTYLINE HCL',
 u'cms_prescription_counts.AMLODIPINE BESYLATE': u'AMLODIPINE BESYLATE',
 u'cms_prescription_counts.AMLODIPINE BESYLATE-BENAZEPRIL': u'AMLODIPINE BESYLATE-BENAZEPRIL',
 u'cms_prescription_

In [16]:
df1 = df.rename(columns=new_names_dict, inplace=True)

In [19]:
df.head()

,ABACAVIR,ABILIFY,ACTONEL,ACYCLOVIR,ADVAIR DISKUS,ALBUTEROL SULFATE,ALENDRONATE SODIUM,ALLOPURINOL,ALPRAZOLAM,AMANTADINE,...,ZIPRASIDONE HCL,ZOLPIDEM TARTRATE,npi,brand_name_rx_count,gender,generic_rx_count,region,settlement_type,specialty,years_practicing
0,NaN,11.0,NaN,NaN,NaN,NaN,NaN,86.0,45.0,NaN,...,NaN,41.0,1295763035,384,M,2287,South,non-urban,Nephrology,7
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1992715205,0,M,103,South,non-urban,General Practice,7
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1578587630,0,M,112,Midwest,non-urban,General Practice,7
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1932278405,0,M,63,South,non-urban,General Practice,7
4,NaN,NaN,16.0,NaN,NaN,NaN,NaN,19.0,NaN,NaN,...,NaN,35.0,1437366804,316,M,1035,West,non-urban,Nephrology,6


In [ ]:
i=0
label_new = ''
new_column_list=[]

len_cms = len('cms_prescription_counts')
len_pv = len('provider_variables')

for label in [df.columns]:
    #print label
    
    label_striped = label.strip()
        
    if(label_striped.startswith('cms_prescription_counts.')):
        label_new = label_striped.replaced('cms_prescription_counts.', '', 1)
        new_column_list.append(label_new)
        
    elif(label_striped.startswith('provider_variables.')):
        label_new = label_striped.replaced('provider_variables.', '', 1)
        new_column_list.append(label_new)
        
    elif (lable_striped == 'npi'):
        print "It's npi, do nothing"
    
    else:
        print "INVALDI COLUMN NAME FOUND [" + label_striped+"]"
        i = i+1
        
    
    
    #if((label_striped =='') or (label_striped.isspace()) or (label_striped.isnumeric())):
    #    i=i+1 #count fro invalide names
    
    
    print "total i is ", i
    

**Uniqueness of npi**

</br>Check for dupliate values of "npi".

In [ ]:
df_npi = pd.DataFrame(df["npi"],columns=['npi']) 
print df_npi.columns.values

In [ ]:
df_npi_dulicates= df_npi.duplicated("npi",keep=False)

In [ ]:
df_npi_dulicates.count()
df_npi_dulicates

In [ ]:
df_npi.describe()
#idx = df_npi.groupby('npi')